本文内容主要来自：
- http://blog.csdn.net/victoriaw/article/details/72872036

In [3]:
import torch
import torch.nn as nn
import torch.nn.init as init

PyTorch提供了多种参数初始化函数：
- torch.nn.init.constant(tensor, val)
- torch.nn.init.normal(tensor, mean=0, std=1)
- torch.nn.init.xavier_uniform(tensor, gain=1)

In [4]:
conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
init.xavier_uniform(conv1.weight)
init.constant(conv1.bias, 0.1)
print(conv1.weight, conv1.bias)

(Parameter containing:
(0 ,0 ,.,.) = 
1.00000e-02 *
  1.4966  0.9459 -4.0601  ...  -2.6735  1.8151 -3.2782
  2.7360  2.3130 -1.3590  ...   0.9779 -0.3665  4.2067
  3.0140 -3.3658 -3.2186  ...   3.4620 -1.2174  0.8918
           ...             ⋱             ...          
 -1.0376  0.8901  2.2509  ...  -3.2813 -2.1093 -0.7652
 -0.7438  0.9604  4.2652  ...   0.2865 -1.7738 -1.3023
 -4.0031 -2.0784 -0.4711  ...   2.6196 -1.7623 -3.6635

(0 ,1 ,.,.) = 
1.00000e-02 *
 -0.0641 -2.6989  1.1737  ...  -1.0922  1.6751  1.5942
 -3.0541  1.0181  3.3403  ...   3.5644  0.0527 -2.9716
  3.1716  3.2811 -0.9251  ...   1.7270  3.6800  2.9606
           ...             ⋱             ...          
  1.1953  3.4653 -3.7302  ...  -0.6696  2.6366  2.1886
  0.0554 -0.0215 -1.9935  ...   0.5594 -0.3820  1.8665
 -2.8589  0.8501 -0.8863  ...  -4.0536  3.0960 -1.8935

(0 ,2 ,.,.) = 
1.00000e-02 *
 -0.5180 -2.5176  0.9794  ...  -2.9021 -2.7209 -1.3858
  0.2433 -1.2021  2.3269  ...   2.1010 -1.7316  0.0584
 -1.2842

上面的语句是对网络的某一层参数进行初始化。如何对整个网络的参数进行初始化定制呢？

In [5]:
def weights_init(m):
    classname=m.__class__.__name__
    if classname.find('Conv') != -1:
        xavier(m.weight.data)
        xavier(m.bias.data)

不建议访问以下划线为前缀的成员，他们是内部的，如果有改变不会通知用户。更推荐的一种方法是检查某个module是否是某种类型：

In [ ]:
def weight_init(m):
# 使用isinstance来判断m属于什么类型
    if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
    elif isinstance(m, nn.BatchNorm2d):
# m中的weight，bias其实都是Variable，为了能学习参数以及后向传播
        m.weight.data.fill_(1)
        m.bias.data.zero_()

net = Net()
net.apply(weights_init) #apply函数会递归地搜索网络内的所有module并把参数表示的函数应用到所有的module上

### Finetune
往往在加载了预训练模型的参数之后，我们需要finetune模型，可以使用不同的方式finetune。
##### 局部微调
有时候我们加载了训练模型后，只想调节最后的几层，其他层不训练。其实不训练也就意味着不进行梯度计算，PyTorch中提供的requires_grad使得对训练的控制变得非常简单。

In [8]:
import torchvision
import torch.optim as optim
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# 替换最后的全连接层， 改为训练100类
# 新构造的模块的参数默认requires_grad为True
model.fc = nn.Linear(512, 100)

# 只优化最后的分类层
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

##### 全局微调
有时候我们需要对全局都进行finetune，只不过我们希望改换过的层和其他层的学习速率不一样，这时候我们可以把其他层和新层在optimizer中单独赋予不同的学习速率。比如：

In [11]:
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 100)
ignored_params = list(map(id, model.fc.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,
                     model.parameters())

optimizer = torch.optim.SGD([
            {'params': base_params},
            {'params': model.fc.parameters(), 'lr': 1e-2}
            ], lr=1e-3, momentum=0.9)

其中base_params使用1e-3来训练，model.fc.parameters使用1e-2来训练，momentum是二者共有的。